In [1]:
from huggingface_hub import login
login()

Token has not been saved to git credential helper.


In [15]:
import pandas as pd
import numpy as np
from tqdm import tqdm


In [16]:
'''
df_train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv", index_col=0)
df_test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv", index_col=0)
'''

'\ndf_train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv", index_col=0)\ndf_test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv", index_col=0)\n'

In [17]:
df_train = pd.read_csv("/media/yanncauchepin/ExternalDisk/Datasets/NaturalLanguageProcessing/disaster_tweets/train.csv", index_col=0)
df_test = pd.read_csv("/media/yanncauchepin/ExternalDisk/Datasets/NaturalLanguageProcessing/disaster_tweets/test.csv", index_col=0)

In [18]:
print(f"Length - train {len(df_train)} - test {len(df_test)}")

Length - train 7613 - test 3263


In [19]:
keywords = pd.concat([
    pd.DataFrame(df_train["keyword"].value_counts()).rename(columns={"count":"train"}),
    pd.DataFrame(df_test["keyword"].value_counts()).rename(columns={"count":"test"})
    ],
    axis=1
)
keywords.head()

,train,test
keyword,,
fatalities,45,5
deluge,42,8
armageddon,42,8
sinking,41,9
damage,41,9


In [20]:
locations = pd.concat([
    pd.DataFrame(df_train["location"].value_counts()).rename(columns={"count":"train"}),
    pd.DataFrame(df_test["location"].value_counts()).rename(columns={"count":"test"})
    ],
    axis=1
)
locations.head()

,train,test
location,,
USA,104.0,37.0
New York,71.0,38.0
United States,50.0,15.0
London,45.0,13.0
Canada,29.0,13.0


In [21]:
df_train_full = df_train.copy()
df_train_full['text'] = df_train_full.apply(lambda row: f"{row['location'] or ''} {row['keyword'] or ''} {row['text']}".strip(), axis=1)
df_test_full = df_test.copy()
df_test_full['text'] = df_test_full.apply(lambda row: f"{row['location'] or ''} {row['keyword'] or ''} {row['text']}".strip(), axis=1)


In [22]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words='english')
train_texts_vec = vectorizer.fit_transform(df_train['text'])
train_texts_vec.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [23]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
train_texts_tfidf = tfidf.fit_transform(train_texts_vec)
train_texts_tfidf = train_texts_tfidf.todense()

In [24]:
test_texts_vec = vectorizer.transform(df_test['text'])
test_texts_vec.todense()
test_texts_tfdif = tfidf.transform(test_texts_vec)
test_texts_tfdif = test_texts_tfdif.todense()
X_test = np.asarray(test_texts_tfdif)

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_texts_tfidf, df_train['target'], test_size = 0.2, stratify=df_train['target'], random_state = 0)

In [26]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_valid = np.asarray(X_valid)
y_valid = np.asarray(y_valid)

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_full = CountVectorizer(stop_words='english')
train_full_texts_vec = vectorizer_full.fit_transform(df_train_full['text'])
train_full_texts_vec.todense()

from sklearn.feature_extraction.text import TfidfTransformer
tfidf_full = TfidfTransformer()
train_full_texts_tfidf = tfidf_full.fit_transform(train_full_texts_vec)
train_full_texts_tfidf = train_full_texts_tfidf.todense()

X_full_train = np.asarray(train_full_texts_tfidf)
y_full_train = np.asarray(df_train_full['target'])

test_full_texts_vec = vectorizer_full.transform(df_test_full['text'])
test_full_texts_vec.todense()
test_full_texts_tfdif = tfidf_full.transform(test_full_texts_vec)
test_full_texts_tfdif = test_full_texts_tfdif.todense()
X_full_test = np.asarray(test_full_texts_tfdif)

In [28]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

def evaluate_classifier(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    cm = confusion_matrix(y_true, y_pred)
    
    metrics_df = pd.DataFrame({
        'Metric': ['F1 Score', 'Precision', 'Recall'],
        'Value': [f1, precision, recall]
    })
    
    cm_df = pd.DataFrame(cm, columns=['Predicted Negative', 'Predicted Positive'], index=['Actual Negative', 'Actual Positive'])
    
    return metrics_df, cm_df

In [16]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train, y_train)
y_pred = naive_bayes_classifier.predict(X_valid)

naive_bayes_classifier_assessement = evaluate_classifier(y_valid, y_pred)
print(naive_bayes_classifier_assessement)

(      Metric     Value
0   F1 Score  0.796308
1  Precision  0.807306
2     Recall  0.801051,                  Predicted Negative  Predicted Positive
Actual Negative                 790                  79
Actual Positive                 224                 430)


In [17]:
naive_bayes_classifier_full = MultinomialNB()
naive_bayes_classifier_full.fit(X_full_train, y_full_train)
y_full_pred = naive_bayes_classifier_full.predict(X_full_test)

naive_bayes_classifier_full_submission = pd.DataFrame({
    'id': df_test_full.index,
    'target': y_full_pred
})

In [18]:
'''
from sklearn.svm import SVC
svc_classifier = SVC(kernel='linear')
svc_classifier.fit(X_train, y_train)
y_pred = svc_classifier.predict(X_valid)

svc_classifier_assessement = evaluate_classifier(y_valid, y_pred)
print(svc_classifier_assessement)
'''

"\nfrom sklearn.svm import SVC\nsvc_classifier = SVC(kernel='linear')\nsvc_classifier.fit(X_train, y_train)\ny_pred = svc_classifier.predict(X_valid)\n\nsvc_classifier_assessement = evaluate_classifier(y_valid, y_pred)\nprint(svc_classifier_assessement)\n"

In [19]:
from sklearn.linear_model import LogisticRegression
logistic_regression_classifier = LogisticRegression()
logistic_regression_classifier.fit(X_train, y_train)
y_pred = logistic_regression_classifier.predict(X_valid)

logistic_regression_classifier_assessement = evaluate_classifier(y_valid, y_pred)
print(logistic_regression_classifier_assessement)

(      Metric     Value
0   F1 Score  0.794483
1  Precision  0.810064
2     Recall  0.800394,                  Predicted Negative  Predicted Positive
Actual Negative                 801                  68
Actual Positive                 236                 418)


In [21]:
logistic_regression_classifier_full = LogisticRegression()
logistic_regression_classifier_full.fit(X_full_train, y_full_train)
y_full_pred = logistic_regression_classifier_full.predict(X_full_test)

logistic_regression_classifier_full_submission = pd.DataFrame({
    'id': df_test_full.index,
    'target': y_full_pred
})

In [ ]:
from sklearn.ensemble import RandomForestClassifier
random_forest_classifier = RandomForestClassifier()
random_forest_classifier.fit(X_train, y_train)
y_pred = random_forest_classifier.predict(X_valid)

random_forest_classifier_assessement = evaluate_classifier(y_valid, y_pred)
print(random_forest_classifier_assessement)

(      Metric     Value
0   F1 Score  0.775101
1  Precision  0.802528
2     Recall  0.784636,                  Predicted Negative  Predicted Positive
Actual Negative                 814                  55
Actual Positive                 273                 381)


In [ ]:
random_forest_classifier_full = RandomForestClassifier()
random_forest_classifier_full.fit(X_full_train, y_full_train)
y_full_pred = random_forest_classifier_full.predict(X_full_test)

random_forest_classifier_full_submission = pd.DataFrame({
    'id': df_test_full.index,
    'target': y_full_pred
})

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

neural_network_classifier = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

neural_network_classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = neural_network_classifier.fit(
    X_train, y_train,
    epochs=10,
    batch_size=8,
    validation_data=(X_valid, y_valid),
    callbacks=[early_stopping]
)

y_pred = neural_network_classifier.predict(X_valid)

neural_network_classifier_assessment = evaluate_classifier(y_valid, np.round(y_pred))
print(neural_network_classifier_assessment)

/home/yanncauchepin/Git/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-10-31 20:49:37.365278: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2024-10-31 20:49:37.365795: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:135] retrieving CUDA diagnostic information for host: yanncauchepincomputing
2024-10-31 20:49:37.365813: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:142] hostname: yanncauchepincomputing
2024-10-31 20:49:37.366105: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:166] libcuda reported version is: 550

Epoch 1/10


2024-10-31 20:49:40.182417: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 520402680 exceeds 10% of free system memory.


762/762 ━━━━━━━━━━━━━━━━━━━━ 14s 16ms/step - accuracy: 0.6765 - loss: 0.5968 - val_accuracy: 0.7984 - val_loss: 0.4524
Epoch 2/10
762/762 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - accuracy: 0.8905 - loss: 0.2714 - val_accuracy: 0.7682 - val_loss: 0.5765
Epoch 3/10
762/762 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - accuracy: 0.9630 - loss: 0.0922 - val_accuracy: 0.7630 - val_loss: 0.7429
Epoch 4/10
762/762 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - accuracy: 0.9837 - loss: 0.0401 - val_accuracy: 0.7525 - val_loss: 0.8938
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
(      Metric     Value
0   F1 Score  0.792614
1  Precision  0.807421
2     Recall  0.798424,                  Predicted Negative  Predicted Positive
Actual Negative                 798                  71
Actual Positive                 236                 418)


In [ ]:
X_full_train_, X_full_valid, y_full_train_, y_full_valid = train_test_split(X_full_train, y_full_train, test_size = 0.15, stratify=y_full_train, random_state = 0)

neural_network_classifier_full = Sequential([
    Dense(64, activation='relu', input_shape=(X_full_train_.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

neural_network_classifier_full.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping_full = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history_full = neural_network_classifier_full.fit(
    X_full_train_, y_full_train_,
    epochs=20,
    batch_size=8,
    validation_data=(X_full_valid, y_full_valid),
    callbacks=[early_stopping]
)

y_full_pred = neural_network_classifier_full.predict(X_full_test)

neural_network_classifier_full_submission = pd.DataFrame({
    'id': df_test_full.index,
    'target': np.round(y_full_pred).flatten()
})

/home/yanncauchepin/Git/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
809/809 ━━━━━━━━━━━━━━━━━━━━ 17s 19ms/step - accuracy: 0.6790 - loss: 0.5842 - val_accuracy: 0.8082 - val_loss: 0.4394
Epoch 2/20
809/809 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - accuracy: 0.8946 - loss: 0.2580 - val_accuracy: 0.7723 - val_loss: 0.5667
Epoch 3/20
809/809 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - accuracy: 0.9689 - loss: 0.0890 - val_accuracy: 0.7627 - val_loss: 0.7224
Epoch 4/20
809/809 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - accuracy: 0.9848 - loss: 0.0359 - val_accuracy: 0.7434 - val_loss: 0.9126
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


In [ ]:
# pip install xgboost catboost

In [ ]:
from xgboost import XGBClassifier

xgb_classifier = XGBClassifier(n_estimators=1000, max_depth=4, learning_rate=0.1, use_label_encoder=False, eval_metric='logloss')
xgb_classifier.fit(X_train, y_train, verbose=1)

y_pred = xgb_classifier.predict(X_valid)

xgb_classifier_assessement = evaluate_classifier(y_valid, y_pred)
print(xgb_classifier_assessement)

/home/yanncauchepin/Git/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [20:58:09] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


(      Metric     Value
0   F1 Score  0.785225
1  Precision  0.794514
2     Recall  0.789888,                  Predicted Negative  Predicted Positive
Actual Negative                 778                  91
Actual Positive                 229                 425)


In [ ]:
xgb_classifier_full = XGBClassifier(n_estimators=1000, max_depth=4, learning_rate=0.1, use_label_encoder=False, eval_metric='logloss')
xgb_classifier_full.fit(X_full_train, y_full_train, verbose=1)

y_full_pred = xgb_classifier_full.predict(X_full_test)

xgb_classifier_full_submission = pd.DataFrame({
    'id': df_test_full.index,
    'target': y_full_pred
})

/home/yanncauchepin/Git/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:00:38] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
from catboost import CatBoostClassifier

catboost_classifier = CatBoostClassifier(iterations=1000, depth=4, learning_rate=0.1, verbose=2)
catboost_classifier.fit(X_train, y_train)

y_pred = catboost_classifier.predict(X_valid)

catboost_classifier_assessement = evaluate_classifier(y_valid, y_pred)
print(catboost_classifier_assessement)

0:	learn: 0.6797045	total: 74.8ms	remaining: 1m 14s
2:	learn: 0.6640753	total: 127ms	remaining: 42.2s
4:	learn: 0.6527238	total: 177ms	remaining: 35.3s
6:	learn: 0.6418404	total: 225ms	remaining: 31.9s
8:	learn: 0.6339999	total: 272ms	remaining: 30s
10:	learn: 0.6280628	total: 324ms	remaining: 29.1s
12:	learn: 0.6209378	total: 389ms	remaining: 29.5s
14:	learn: 0.6162400	total: 437ms	remaining: 28.7s
16:	learn: 0.6115203	total: 486ms	remaining: 28.1s
18:	learn: 0.6076645	total: 538ms	remaining: 27.8s
20:	learn: 0.6030091	total: 586ms	remaining: 27.3s
22:	learn: 0.5995534	total: 636ms	remaining: 27s
24:	learn: 0.5964546	total: 683ms	remaining: 26.7s
26:	learn: 0.5930054	total: 732ms	remaining: 26.4s
28:	learn: 0.5902966	total: 781ms	remaining: 26.2s
30:	learn: 0.5873470	total: 828ms	remaining: 25.9s
32:	learn: 0.5849357	total: 877ms	remaining: 25.7s
34:	learn: 0.5827788	total: 926ms	remaining: 25.5s
36:	learn: 0.5798956	total: 975ms	remaining: 25.4s
38:	learn: 0.5778346	total: 1.02s	rema

In [ ]:
catboost_classifier_full = CatBoostClassifier(iterations=1000, depth=4, learning_rate=0.1, verbose=2)
catboost_classifier_full.fit(X_full_train, y_full_train)

y_full_pred = catboost_classifier_full.predict(X_full_test)

catboost_classifier_full_submission = pd.DataFrame({
    'id': df_test_full.index,
    'target': y_full_pred
})

0:	learn: 0.6830062	total: 28.3ms	remaining: 28.3s
2:	learn: 0.6635779	total: 83.1ms	remaining: 27.6s
4:	learn: 0.6506161	total: 136ms	remaining: 27s
6:	learn: 0.6415673	total: 188ms	remaining: 26.7s
8:	learn: 0.6337147	total: 240ms	remaining: 26.5s
10:	learn: 0.6258401	total: 295ms	remaining: 26.6s
12:	learn: 0.6205170	total: 347ms	remaining: 26.3s
14:	learn: 0.6152465	total: 400ms	remaining: 26.3s
16:	learn: 0.6112210	total: 453ms	remaining: 26.2s
18:	learn: 0.6066602	total: 507ms	remaining: 26.2s
20:	learn: 0.6036622	total: 559ms	remaining: 26.1s
22:	learn: 0.6004812	total: 612ms	remaining: 26s
24:	learn: 0.5975389	total: 665ms	remaining: 25.9s
26:	learn: 0.5946601	total: 718ms	remaining: 25.9s
28:	learn: 0.5914057	total: 771ms	remaining: 25.8s
30:	learn: 0.5874459	total: 824ms	remaining: 25.8s
32:	learn: 0.5840149	total: 880ms	remaining: 25.8s
34:	learn: 0.5819688	total: 936ms	remaining: 25.8s
36:	learn: 0.5796179	total: 989ms	remaining: 25.7s
38:	learn: 0.5774566	total: 1.04s	rema

In [ ]:
# pip install transformers torch

In [ ]:
# pip install datasets

In [ ]:
# !pip install hugging_face_hub

In [29]:
import torch
from transformers import Trainer, TrainingArguments
from datasets import Dataset

2024-11-01 22:38:21.599877: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-01 22:38:21.629607: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-01 22:38:21.639531: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-01 22:38:21.665732: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-01 22:38:23.377331: W tensorflow/compiler/tf2

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

bert_tokenizer_full = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model_full = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

bert_encodings_full = bert_tokenizer_full(list(df_train_full['text']), truncation=True, padding=True, max_length=256)
bert_labels_full = torch.tensor(list(df_train_full['target']))

bert_input_ids_train_full, bert_input_ids_valid_full, bert_token_type_ids_train_full, bert_token_type_ids_valid_full, \
bert_attention_mask_train_full, bert_attention_mask_valid_full, bert_y_train_full, bert_y_valid_full = train_test_split(
    bert_encodings_full['input_ids'], 
    bert_encodings_full['token_type_ids'], 
    bert_encodings_full['attention_mask'], 
    bert_labels_full, 
    test_size=0.15, 
    stratify=bert_labels_full, 
    random_state=0
)

bert_train_encodings_full = {
    'input_ids': torch.tensor(bert_input_ids_train_full),
    'token_type_ids': torch.tensor(bert_token_type_ids_train_full),
    'attention_mask': torch.tensor(bert_attention_mask_train_full)
}

bert_valid_encodings_full = {
    'input_ids': torch.tensor(bert_input_ids_valid_full),
    'token_type_ids': torch.tensor(bert_token_type_ids_valid_full),
    'attention_mask': torch.tensor(bert_attention_mask_valid_full)
}


training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="epoch",
    report_to="none",
    no_cuda=True 
)

bert_train_dataset_full = Dataset.from_dict({
    "input_ids": bert_train_encodings_full['input_ids'],
    "attention_mask": bert_train_encodings_full['attention_mask'],
    "labels": bert_y_train_full
})

bert_valid_dataset_full = Dataset.from_dict({
    "input_ids": bert_valid_encodings_full['input_ids'],
    "attention_mask": bert_valid_encodings_full['attention_mask'],
    "labels": bert_y_valid_full
})

bert_trainer_full = Trainer(
    model=bert_model_full,
    args=training_args,
    train_dataset=bert_train_dataset_full,
    eval_dataset=bert_valid_dataset_full
)

bert_trainer_full.train()

bert_predictions_full = bert_trainer_full.predict(bert_valid_dataset_full)
bert_logits_full = bert_predictions_full.predictions
bert_y_pred_full = np.argmax(bert_logits_full, axis=1)

bert_trainer_full_assessement = evaluate_classifier(bert_y_valid_full.numpy(), bert_y_pred_full)

bert_trainer_full.save_model("disastertweets_bert_model")
bert_trainer_full.push_to_hub("yanncauchepin/kaggle_disastertweets_bert_model")

bert_test_encodings_full = bert_tokenizer_full(list(df_test_full['text']), truncation=True, padding=True, max_length=256)
bert_test_encodings_full = {key: torch.tensor(val) for key, val in bert_test_encodings_full.items()}
bert_test_dataset_full = Dataset.from_dict({
    "input_ids": bert_test_encodings_full['input_ids'],
    "attention_mask": bert_test_encodings_full['attention_mask']
})

bert_test_predictions_full = bert_trainer_full.predict(bert_test_dataset_full)
bert_test_logits_full = bert_test_predictions_full.predictions
bert_test_y_pred_full = np.argmax(bert_test_logits_full, axis=1)

bert_test_submission_full = pd.DataFrame({
    'id': df_test_full.index,
    'target': bert_test_y_pred_full.flatten()
})

bert_trainer_full.save_pretrained("kaggle_disastertweets_bert_trainer")
bert_tokenizer_full.save_pretrained("kaggle_disastertweets_bert_tokenizer")

hf_bert_test_submission_full = Dataset.from_pandas(bert_test_submission_full)
hf_bert_test_submission_full.push_to_hub("yanncauchepin/kaggle_disastertweets_bert_submission_df")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/yanncauchepin/Git/.venv/lib/python3.12/site-packages/transformers/training_args.py:1574: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


  0%|          | 0/4854 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# !pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 1.3 MB/s eta 0:00:00a 0:00:01


In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

distill_bert_tokenizer_full = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
distill_bert_model_full = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

distill_bert_encodings_full = distill_bert_tokenizer_full(list(df_train_full['text']), truncation=True, padding=True, max_length=256)
distill_bert_labels_full = torch.tensor(list(df_train_full['target']))

distill_bert_input_ids_train_full, distill_bert_input_ids_valid_full, \
distill_bert_attention_mask_train_full, distill_bert_attention_mask_valid_full, distill_bert_y_train_full, distill_bert_y_valid_full = train_test_split(
    distill_bert_encodings_full['input_ids'], 
    distill_bert_encodings_full['attention_mask'], 
    distill_bert_labels_full, 
    test_size=0.15, 
    stratify=distill_bert_labels_full, 
    random_state=0
)

distill_bert_train_encodings_full = {
    'input_ids': torch.tensor(distill_bert_input_ids_train_full),
    'attention_mask': torch.tensor(distill_bert_attention_mask_train_full)
}

distill_bert_valid_encodings_full = {
    'input_ids': torch.tensor(distill_bert_input_ids_valid_full),
    'attention_mask': torch.tensor(distill_bert_attention_mask_valid_full)
}


training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="epoch",
    report_to="none",
    no_cuda=True 
)

distill_bert_train_dataset_full = Dataset.from_dict({
    "input_ids": distill_bert_train_encodings_full['input_ids'],
    "attention_mask": distill_bert_train_encodings_full['attention_mask'],
    "labels": distill_bert_y_train_full
})

distill_bert_valid_dataset_full = Dataset.from_dict({
    "input_ids": distill_bert_valid_encodings_full['input_ids'],
    "attention_mask": distill_bert_valid_encodings_full['attention_mask'],
    "labels": distill_bert_y_valid_full
})

distill_bert_trainer_full = Trainer(
    model=distill_bert_model_full,
    args=training_args,
    train_dataset=distill_bert_train_dataset_full,
    eval_dataset=distill_bert_valid_dataset_full
)

distill_bert_trainer_full.train()

distill_bert_predictions_full = distill_bert_trainer_full.predict(distill_bert_valid_dataset_full)
distill_bert_logits_full = distill_bert_predictions_full.predictions
distill_bert_y_pred_full = np.argmax(distill_bert_logits_full, axis=1)

distill_bert_trainer_full_assessement = evaluate_classifier(distill_bert_y_valid_full.numpy(), distill_bert_y_pred_full)

distill_bert_test_encodings_full = distill_bert_tokenizer_full(list(df_test_full['text']), truncation=True, padding=True, max_length=256)
distill_bert_test_encodings_full = {key: torch.tensor(val) for key, val in distill_bert_test_encodings_full.items()}
distill_bert_test_dataset_full = Dataset.from_dict({
    "input_ids": distill_bert_test_encodings_full['input_ids'],
    "attention_mask": distill_bert_test_encodings_full['attention_mask']
})

distill_bert_test_predictions_full = distill_bert_trainer_full.predict(distill_bert_test_dataset_full)
distill_bert_test_logits_full = distill_bert_test_predictions_full.predictions
distill_bert_test_y_pred_full = np.argmax(distill_bert_test_logits_full, axis=1)

distill_bert_test_submission_full = pd.DataFrame({
    'id': df_test_full.index,
    'target': distill_bert_test_y_pred_full.flatten()
})

distill_bert_trainer_full.save_pretrained("disastertweets_distill_bert_model")
distill_bert_tokenizer_full.save_pretrained("disastertweets_distill_bert_tokenizer")

distill_bert_trainer_full.model.push_to_hub("yanncauchepin/kaggle_disastertweets_distill_bert_model")
distill_bert_tokenizer_full.push_to_hub("yanncauchepin/kaggle_disastertweets_distill_bert_tokenizer")

hf_distill_bert_test_submission_full = Dataset.from_pandas(distill_bert_test_submission_full)
hf_distill_bert_test_submission_full.push_to_hub("yanncauchepin/kaggle_disastertweets_distill_bert_submission_df")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/yanncauchepin/Git/.venv/lib/python3.12/site-packages/transformers/training_args.py:1574: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


  0%|          | 0/4854 [00:00<?, ?it/s]

{'loss': 0.7026, 'grad_norm': 3.5670106410980225, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}
{'loss': 0.6921, 'grad_norm': 1.994215726852417, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}
{'loss': 0.6435, 'grad_norm': 4.213106155395508, 'learning_rate': 3e-06, 'epoch': 0.02}
{'loss': 0.7004, 'grad_norm': 3.5397398471832275, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.02}
{'loss': 0.6915, 'grad_norm': 2.5893571376800537, 'learning_rate': 5e-06, 'epoch': 0.03}
{'loss': 0.657, 'grad_norm': 2.7244741916656494, 'learning_rate': 6e-06, 'epoch': 0.04}
{'loss': 0.6811, 'grad_norm': 3.9375064373016357, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.04}
{'loss': 0.656, 'grad_norm': 2.504549980163574, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.05}
{'loss': 0.6243, 'grad_norm': 3.951978921890259, 'learning_rate': 9e-06, 'epoch': 0.06}
{'loss': 0.6251, 'grad_norm': 5.363071441650391, 'learning_rate': 1e-05, 'epoch': 0.06}
{'loss': 0.5678, 'grad_norm': 4.704

  0%|          | 0/286 [00:00<?, ?it/s]

{'eval_loss': 0.4842619001865387, 'eval_runtime': 53.463, 'eval_samples_per_second': 21.361, 'eval_steps_per_second': 5.349, 'epoch': 1.0}
{'loss': 0.3562, 'grad_norm': 10.87424373626709, 'learning_rate': 3.713826366559486e-05, 'epoch': 1.0}
{'loss': 0.4723, 'grad_norm': 29.80830192565918, 'learning_rate': 3.702342673403767e-05, 'epoch': 1.01}
{'loss': 0.6169, 'grad_norm': 1.1386029720306396, 'learning_rate': 3.690858980248048e-05, 'epoch': 1.01}
{'loss': 0.4432, 'grad_norm': 0.7023907899856567, 'learning_rate': 3.679375287092329e-05, 'epoch': 1.02}
{'loss': 0.447, 'grad_norm': 4.798180103302002, 'learning_rate': 3.6678915939366105e-05, 'epoch': 1.03}
{'loss': 0.3589, 'grad_norm': 10.056122779846191, 'learning_rate': 3.656407900780892e-05, 'epoch': 1.03}
{'loss': 0.9171, 'grad_norm': 36.724205017089844, 'learning_rate': 3.644924207625172e-05, 'epoch': 1.04}
{'loss': 0.5244, 'grad_norm': 7.010728359222412, 'learning_rate': 3.6334405144694535e-05, 'epoch': 1.04}
{'loss': 0.4184, 'grad_no

...

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/yanncauchepin/kaggle_disastertweets_distill_bert_submission_df/commit/e306c0819327d23bd42456978839b3ea9279dabd', commit_message='Upload dataset', commit_description='', oid='e306c0819327d23bd42456978839b3ea9279dabd', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/yanncauchepin/kaggle_disastertweets_distill_bert_submission_df', endpoint='https://huggingface.co', repo_type='dataset', repo_id='yanncauchepin/kaggle_disastertweets_distill_bert_submission_df'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

robert_tokenizer_full = RobertaTokenizer.from_pretrained("roberta-base")
robert_model_full = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

robert_encodings_full = robert_tokenizer_full(list(df_train_full['text']), truncation=True, padding=True, max_length=256)
robert_labels_full = torch.tensor(list(df_train_full['target']))

robert_input_ids_train_full, robert_input_ids_valid_full, \
robert_attention_mask_train_full, robert_attention_mask_valid_full, robert_y_train_full, robert_y_valid_full = train_test_split(
    robert_encodings_full['input_ids'], 
    robert_encodings_full['attention_mask'], 
    robert_labels_full, 
    test_size=0.15, 
    stratify=robert_labels_full, 
    random_state=0
)

robert_train_encodings_full = {
    'input_ids': torch.tensor(robert_input_ids_train_full),
    'attention_mask': torch.tensor(robert_attention_mask_train_full)
}

robert_valid_encodings_full = {
    'input_ids': torch.tensor(robert_input_ids_valid_full),
    'attention_mask': torch.tensor(robert_attention_mask_valid_full)
}


training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="epoch",
    report_to="none",
    no_cuda=True 
)

robert_train_dataset_full = Dataset.from_dict({
    "input_ids": robert_train_encodings_full['input_ids'],
    "attention_mask": robert_train_encodings_full['attention_mask'],
    "labels": robert_y_train_full
})

robert_valid_dataset_full = Dataset.from_dict({
    "input_ids": robert_valid_encodings_full['input_ids'],
    "attention_mask": robert_valid_encodings_full['attention_mask'],
    "labels": robert_y_valid_full
})

robert_trainer_full = Trainer(
    model=robert_model_full,
    args=training_args,
    train_dataset=robert_train_dataset_full,
    eval_dataset=robert_valid_dataset_full
)

robert_trainer_full.train()

robert_predictions_full = robert_trainer_full.predict(robert_valid_dataset_full)
robert_logits_full = robert_predictions_full.predictions
robert_y_pred_full = np.argmax(robert_logits_full, axis=1)

robert_trainer_full_assessement = evaluate_classifier(robert_y_valid_full.numpy(), robert_y_pred_full)

robert_test_encodings_full = robert_tokenizer_full(list(df_test_full['text']), truncation=True, padding=True, max_length=256)
robert_test_encodings_full = {key: torch.tensor(val) for key, val in robert_test_encodings_full.items()}
robert_test_dataset_full = Dataset.from_dict({
    "input_ids": robert_test_encodings_full['input_ids'],
    "attention_mask": robert_test_encodings_full['attention_mask']
})

robert_test_predictions_full = robert_trainer_full.predict(robert_test_dataset_full)
robert_test_logits_full = robert_test_predictions_full.predictions
robert_test_y_pred_full = np.argmax(robert_test_logits_full, axis=1)

robert_test_submission_full = pd.DataFrame({
    'id': df_test_full.index,
    'target': robert_test_y_pred_full.flatten()
})

robert_trainer_full.model.save_pretrained("kaggle_disastertweets_robert_model")
robert_tokenizer_full.save_pretrained("kaggle_disastertweets_robert_tokenizer")

robert_trainer_full.model.push_to_hub("yanncauchepin/kaggle_disastertweets_robert_model")
robert_tokenizer_full.push_to_hub("yanncauchepin/kaggle_disastertweets_robert_tokenizer")

hf_robert_test_submission_full = Dataset.from_pandas(robert_test_submission_full)
hf_robert_test_submission_full.push_to_hub("yanncauchepin/kaggle_disastertweets_robert_submission_df")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/yanncauchepin/Git/.venv/lib/python3.12/site-packages/transformers/training_args.py:1574: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


  0%|          | 0/4854 [00:00<?, ?it/s]

{'loss': 0.7047, 'grad_norm': 4.763729572296143, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}
{'loss': 0.6909, 'grad_norm': 2.3180086612701416, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}
{'loss': 0.6959, 'grad_norm': 4.462715148925781, 'learning_rate': 3e-06, 'epoch': 0.02}
{'loss': 0.693, 'grad_norm': 4.434909820556641, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.02}
{'loss': 0.6917, 'grad_norm': 3.107389211654663, 'learning_rate': 5e-06, 'epoch': 0.03}
{'loss': 0.6905, 'grad_norm': 5.843577861785889, 'learning_rate': 6e-06, 'epoch': 0.04}
{'loss': 0.6944, 'grad_norm': 6.763678073883057, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.04}
{'loss': 0.696, 'grad_norm': 4.3050079345703125, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.05}
{'loss': 0.6812, 'grad_norm': 10.309606552124023, 'learning_rate': 9e-06, 'epoch': 0.06}
{'loss': 0.72, 'grad_norm': 4.82548713684082, 'learning_rate': 1e-05, 'epoch': 0.06}
{'loss': 0.6682, 'grad_norm': 1.92471420

  0%|          | 0/286 [00:00<?, ?it/s]

{'eval_loss': 0.5335590839385986, 'eval_runtime': 99.6202, 'eval_samples_per_second': 11.464, 'eval_steps_per_second': 2.871, 'epoch': 1.0}
{'loss': 0.4928, 'grad_norm': 1.1166706085205078, 'learning_rate': 3.713826366559486e-05, 'epoch': 1.0}
{'loss': 0.3916, 'grad_norm': 1.8034169673919678, 'learning_rate': 3.702342673403767e-05, 'epoch': 1.01}
{'loss': 0.6014, 'grad_norm': 2.653855800628662, 'learning_rate': 3.690858980248048e-05, 'epoch': 1.01}
{'loss': 0.47, 'grad_norm': 2.2010912895202637, 'learning_rate': 3.679375287092329e-05, 'epoch': 1.02}
{'loss': 0.5237, 'grad_norm': 0.9264560341835022, 'learning_rate': 3.6678915939366105e-05, 'epoch': 1.03}
{'loss': 0.5434, 'grad_norm': 3.8087637424468994, 'learning_rate': 3.656407900780892e-05, 'epoch': 1.03}
{'loss': 0.6922, 'grad_norm': 4.619762897491455, 'learning_rate': 3.644924207625172e-05, 'epoch': 1.04}
{'loss': 0.5363, 'grad_norm': 2.6939892768859863, 'learning_rate': 3.6334405144694535e-05, 'epoch': 1.04}
{'loss': 0.3662, 'grad_

...

  0%|          | 0/816 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/yanncauchepin/kaggle_disastertweets_robert_submission_df/commit/f6416953df4cb11f0adc0f798db5431b90e714e3', commit_message='Upload dataset', commit_description='', oid='f6416953df4cb11f0adc0f798db5431b90e714e3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/yanncauchepin/kaggle_disastertweets_robert_submission_df', endpoint='https://huggingface.co', repo_type='dataset', repo_id='yanncauchepin/kaggle_disastertweets_robert_submission_df'), pr_revision=None, pr_num=None)

In [ ]:
# !pip install sentencepiece

In [ ]:
import sentencepiece
from transformers import AlbertTokenizer, AlbertForSequenceClassification

albert_tokenizer_full = AlbertTokenizer.from_pretrained("albert-base-v2")
albert_model_full = AlbertForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)

albert_encodings_full = albert_tokenizer_full(list(df_train_full['text']), truncation=True, padding=True, max_length=256)
albert_labels_full = torch.tensor(list(df_train_full['target']))

albert_input_ids_train_full, albert_input_ids_valid_full, albert_token_type_ids_train_full, albert_token_type_ids_valid_full, \
albert_attention_mask_train_full, albert_attention_mask_valid_full, albert_y_train_full, albert_y_valid_full = train_test_split(
    albert_encodings_full['input_ids'], 
    albert_encodings_full['token_type_ids'], 
    albert_encodings_full['attention_mask'], 
    albert_labels_full, 
    test_size=0.15, 
    stratify=albert_labels_full, 
    random_state=0
)

albert_train_encodings_full = {
    'input_ids': torch.tensor(albert_input_ids_train_full),
    'token_type_ids': torch.tensor(albert_token_type_ids_train_full),
    'attention_mask': torch.tensor(albert_attention_mask_train_full)
}

albert_valid_encodings_full = {
    'input_ids': torch.tensor(albert_input_ids_valid_full),
    'token_type_ids': torch.tensor(albert_token_type_ids_valid_full),
    'attention_mask': torch.tensor(albert_attention_mask_valid_full)
}


training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="epoch",
    report_to="none",
    no_cuda=True 
)

albert_train_dataset_full = Dataset.from_dict({
    "input_ids": albert_train_encodings_full['input_ids'],
    "attention_mask": albert_train_encodings_full['attention_mask'],
    "labels": albert_y_train_full
})

albert_valid_dataset_full = Dataset.from_dict({
    "input_ids": albert_valid_encodings_full['input_ids'],
    "attention_mask": albert_valid_encodings_full['attention_mask'],
    "labels": albert_y_valid_full
})

albert_trainer_full = Trainer(
    model=albert_model_full,
    args=training_args,
    train_dataset=albert_train_dataset_full,
    eval_dataset=albert_valid_dataset_full
)

albert_trainer_full.train()

albert_predictions_full = albert_trainer_full.predict(albert_valid_dataset_full)
albert_logits_full = albert_predictions_full.predictions
albert_y_pred_full = np.argmax(albert_logits_full, axis=1)

albert_trainer_full_assessement = evaluate_classifier(albert_y_valid_full.numpy(), albert_y_pred_full)

albert_test_encodings_full = albert_tokenizer_full(list(df_test_full['text']), truncation=True, padding=True, max_length=256)
albert_test_encodings_full = {key: torch.tensor(val) for key, val in albert_test_encodings_full.items()}
albert_test_dataset_full = Dataset.from_dict({
    "input_ids": albert_test_encodings_full['input_ids'],
    "attention_mask": albert_test_encodings_full['attention_mask']
})

albert_test_predictions_full = albert_trainer_full.predict(albert_test_dataset_full)
albert_test_logits_full = albert_test_predictions_full.predictions
albert_test_y_pred_full = np.argmax(albert_test_logits_full, axis=1)

albert_test_submission_full = pd.DataFrame({
    'id': df_test_full.index,
    'target': albert_test_y_pred_full.flatten()
})

albert_trainer_full.model.save_pretrained("kaggle_disastertweets_albert_model")
albert_tokenizer_full.save_pretrained("kaggle_disastertweets_albert_tokenizer")

albert_trainer_full.model.push_to_hub("yanncauchepin/kaggle_disastertweets_albert_model")
albert_tokenizer_full.push_to_hub("yanncauchepin/kaggle_disastertweets_albert_tokenizer")

hf_albert_test_submission_full = Dataset.from_pandas(albert_test_submission_full)
hf_albert_test_submission_full.push_to_hub("yanncauchepin/kaggle_disastertweets_albert_submission_df")

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/yanncauchepin/Git/.venv/lib/python3.12/site-packages/transformers/training_args.py:1574: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


  0%|          | 0/4854 [00:00<?, ?it/s]

{'loss': 0.7353, 'grad_norm': 29.566905975341797, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}
{'loss': 0.7345, 'grad_norm': 30.92800521850586, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}
{'loss': 0.776, 'grad_norm': 34.36389923095703, 'learning_rate': 3e-06, 'epoch': 0.02}
{'loss': 0.6873, 'grad_norm': 20.674179077148438, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.02}
{'loss': 0.7031, 'grad_norm': 21.924238204956055, 'learning_rate': 5e-06, 'epoch': 0.03}
{'loss': 0.643, 'grad_norm': 21.9400691986084, 'learning_rate': 6e-06, 'epoch': 0.04}
{'loss': 0.6528, 'grad_norm': 122.38347625732422, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.04}
{'loss': 0.63, 'grad_norm': 65.72347259521484, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.05}
{'loss': 0.5723, 'grad_norm': 21.250648498535156, 'learning_rate': 9e-06, 'epoch': 0.06}
{'loss': 0.7273, 'grad_norm': 66.90182495117188, 'learning_rate': 1e-05, 'epoch': 0.06}
{'loss': 0.5421, 'grad_norm': 37.23288